In [15]:
import requests
import pandas as pd

In [18]:
# Banned Player List 가져오기
url = "https://api.gametools.network/"
# params = {Key:Value}
response = requests.get(url+"bfban/banned_players/")

if response.status_code == 200:
    # 성공적으로 데이터를 받아옴
    data = response.json() # JSON 응답을 파이썬 딕셔너리로 변환
    data_df = pd.DataFrame(data, columns=["personaids"])
    data_df.to_csv("./banned_players.csv", index=False)
else:
    # 오류 처리
    print(f'Error: {response.status_code}')

In [23]:
# Banned User 정보 가져오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 사용자 ID 목록 (예시)
band_user_ids = ["204752101"]

# 데이터를 저장할 리스트 초기화
data = []

# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.headless = True  # 브라우저가 UI 없이 실행되도록 설정 (백그라운드 실행)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 각 사용자 ID에 대해 반복
for user_id in band_user_ids:
    url = f'https://bfban.com/player/{user_id}'
    driver.get(url)
    
    # 페이지 로딩 대기
    time.sleep(5)  # 실제 웹 페이지에 따라 조정이 필요할 수 있습니다.
    
    # 필요한 데이터 추출
    feature1_elements = driver.find_elements(By.CLASS_NAME, "ivu-tag-text.ivu-tag-color-white")
    feature2_elements = driver.find_elements(By.CSS_SELECTOR, 'img.ivu-tag-text.ivu-tag-color-default')
    feature3_elements = driver.find_elements(By.CSS_SELECTOR, 'span.ivu-tag.ivu-tag-size-default.ivu-tag-warning.ivu-tag-checked')
    feature4_elements = driver.find_elements(By.CSS_SELECTOR, 'code.text-distinguishing-letter.ivu-col')
    
    # 데이터 추가
    for f1, f2, f3, f4 in zip(feature1_elements, feature2_elements, feature3_elements, feature4_elements):
        data.append({
            'UserName': f1.text if f1 else '',
            'ID': user_id,
            'Feature2': f2.get_attribute('src') if f2 else '',
            'Feature3': f3.text if f3 else '',
            'Feature4': f4.text if f4 else '',
        })

# 드라이버 종료
driver.quit()

# 데이터프레임 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
csv_file_path = '/mnt/data/band_user_data_selenium.csv'
df.to_csv(csv_file_path, index=False)

print(f'CSV 파일이 생성되었습니다: {csv_file_path}')


ModuleNotFoundError: No module named 'selenium'